In [2]:
!pip install "snowflake-connector-python[pandas]" snowflake-snowpark-python scikit-learn selenium nltk


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import pandas as pd
import numpy as np
import streamlit as st # Nueva importación para la app Streamlit
from snowflake.snowpark import Session
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata


In [4]:
SNOWFLAKE_CONFIG = {
    "account":  os.getenv("SNOWFLAKE_ACCOUNT", "FPSYQCG-JS43936"),
    "user":     os.getenv("SNOWFLAKE_USER", "ARONDON"),
    "password": os.getenv("SNOWFLAKE_PASSWORD", "ZDj3qMyj9xaEM6J"),  # ⚠️ mueve esto a variables de entorno
    "role":     os.getenv("SNOWFLAKE_ROLE", "readonly"),
    "warehouse": os.getenv("SNOWFLAKE_WH", "compute_wh"),
    "database":  os.getenv("SNOWFLAKE_DB", "TUI_TFM"),
    "schema":    os.getenv("SNOWFLAKE_SCHEMA", "PROCESSED"),
}

In [5]:
def load_data_from_snowflake():
    """Abre sesión, descarga tablas necesarias y cierra la sesión."""
    try:
        with Session.builder.configs(SNOWFLAKE_CONFIG).create() as sf_session:
            print("✅ Conectado a Snowflake")
            master_forecasted_df = sf_session.table("TUI_TFM.PROCESSED.MASTER_FORECASTED").to_pandas()
            experiencis_df = sf_session.table("TUI_TFM.PROCESSED.EXP_TUI").to_pandas()

            return experiencis_df, master_forecasted_df
    except Exception as e:
        raise RuntimeError(f"❌ Error al cargar desde Snowflake: {e}")

In [81]:
# =========================
# Helpers
# =========================

def ask(msg, default=None, cast=str, valid=None):
    while True:
        val = input(f"{msg} [{default}]: ").strip().lower() or str(default).lower()
        try:
            val = cast(val)
        except Exception:
            print("⚠️ Entrada no válida.")
            continue
        if valid and not valid(val):
            print("⚠️ Valor fuera de rango o no permitido.")
            continue
        return val
    

def normalize(text: str) -> str:
    return ''.join(
        c for c in unicodedata.normalize("NFD", text)
        if unicodedata.category(c) != "Mn"
    ).lower().strip()  

def validar_provincia(raw_input: str, df_provincias: pd.Series) -> str:

    PROV_ALIASES = {
        "las palmas": "Palmas, Las",
        "palmas": "Palmas, Las",
        "coruna": "Coruña, A",
        "a coruna": "Coruña, A",
        "la coruna": "Coruña, A",
        "baleares": "Balears, Illes",
        "balears": "Balears, Illes",
        "islas baleares": "Balears, Illes",
        "illes balears": "Balears, Illes",
        "la rioja": "Rioja, La",
        "rioja": "Rioja, La",
    }

    norm_input = normalize(raw_input)

    # Primero intentamos con aliases
    if norm_input in PROV_ALIASES:
        return PROV_ALIASES[norm_input].lower()

    # Si no está en aliases, buscamos entre las provincias del DataFrame
    prov_map = {normalize(p): p for p in df_provincias.unique()}
    if norm_input in prov_map:
        return prov_map[norm_input].lower()

    # Finalmente, buscamos coincidencias parciales: si el input está contenido en algún nombre
    for norm_name, official_name in prov_map.items():
        if norm_input in norm_name:  # coincidencia parcial
            return official_name.lower()

    raise ValueError(f"Provincia '{raw_input}' no encontrada")

def norm_0_1(series: pd.Series) -> pd.Series:
    """Normaliza una serie a [0,1] de forma segura."""
    s = series.astype(float)
    return (s - s.min()) / (s.max() - s.min()) if s.max() > s.min() else 0 * s

def normalize_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Normaliza las columnas indicadas del DataFrame usando norm_0_1.
    Devuelve un DataFrame con las columnas normalizadas, renombradas con '_index'.
    """
    df_norm = pd.DataFrame(index=df.index)  # Nuevo DF solo con índices
    for c in cols:
        if c in df.columns:
            df_norm[f"{c}_index"] = norm_0_1(df[c])
    return df_norm

def _geo_accept_set(t: str) -> set:

    if t == "playa":
        return {"playa", "mixto"}
    if t == "montaña":
        return {"montana", "mixto"}
    if t == "urbano":
        return {"urbano"}                      # mixto NO vale para urbano
    if t == "mixto":
        return {"playa", "montana", "mixto"}   # mixto = playa+montaña
    return set()

def _geo_mask(series_geo: pd.Series, tourism: str) -> pd.Series:
    accept = _geo_accept_set(tourism)
    if not accept:
        return pd.Series(True, index=series_geo.index)
    return series_geo.isin(accept)

def categorize(value, series):
    p25, p75 = series.quantile([0.25, 0.75])
    if value <= p25:
        return "low"
    elif value <= p75:
        return "medium"
    else:
        return "high"
    
def _alts_once(df_slice, base_row, sim_matrix, tourism_type, sim_geo_bonus, 
               top_set, target_temp, crowd_delta, temp_dev, user_drop, require_geo=True):

    pos = {p: i for i, p in enumerate(df_slice["province"].tolist())}        
    i = pos[base_row["province"]]
    candidates = df_slice.copy()
    sim_base = sim_matrix[i]
    
    # bonus similitud por geografía alineada
    if tourism_type:
        cg = candidates["geography"]
        geo_accept = _geo_accept_set(tourism_type)  # devuelve el set de geografía aceptable
        geo_ok = cg.isin(geo_accept)
        sim_adj = sim_base + sim_geo_bonus * geo_ok.astype(float).values
    else:
        sim_adj = sim_base
        geo_ok = pd.Series(True, index=candidates.index)

    mask = (
        (~candidates["province"].isin(top_set)) &
        (candidates["crowd_index"] < base_row["crowd_index"] - float(crowd_delta)) &
        (candidates["user_score"] >= base_row["user_score"] * (1.0 - float(user_drop))) &
        ((candidates["mean_temp"] - float(target_temp)).abs() <= float(temp_dev))
    )

    if require_geo:
        mask = mask & geo_ok

    cand = candidates[mask].copy()
    if cand.empty:
        return cand
    cand["similarity"] = sim_adj[mask]
    return cand.nlargest(3, "similarity")

In [7]:
def build_province_month_features(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Renombra columnas a un estándar, extrae mes, construye eventos y
    genera índices normalizados por mes (demanda/precio/crowd).
    """
    column_map = {
        "NOMBRE_CCAA": "ccaa",                           # si existe
        "NOMBRE_PRO": "province", "PERIODO": "period",
        "TEMP_MED": "mean_temp", "TEMP_MAX": "max_temp", "TEMP_MIN": "min_temp",
        "PRECIPITACION": "rain", "ADR": "price",
        "PROPORCION_OCUPACION_HABIT": "crowd",
        "PROPORCION_OCUPACION_CAMAS": "occ_beds",
        "PROPORCION_OCUPACION_CAMAS_FINDE": "occ_beds_weekend",
        "ESTABLECIMIENTOS_ABIERTOS": "establishments",
        "NUMERO_DE_HABITACIONES": "n_rooms", "NUMERO_DE_CAMAS": "n_beds",
        "EMPLEADOS": "employees", "TURISTAS": "tourists",
        "ALTITUD": "altitude",
        "GEOGRAFIA": "geography",
    }
    df = raw_df.rename(columns=column_map).copy()

    # Formato YYYY-MM -> Period(M)
    df["period_str"] = df["period"].astype(str).str.replace("M", "-") 
    df["year_month"] = pd.to_datetime(df["period_str"], format="%Y-%m", errors="coerce").dt.to_period("M")

    df["geography"] = df["geography"].str.lower()

    # Total de eventos
    event_cols = ["HOBBIES_AND_GAMES", "ARTS_AND_SOCIETY", "SPORTS_AND_WELLNESS", "FESTIVALS", "FOOD", "FAMILY"]
    df["events_total"] = df[event_cols].sum(axis=1)

    feature_cols = [
        "province", "year_month",
        "mean_temp", "max_temp", "min_temp", "rain",
        "events_total", *event_cols, "price", "crowd",
        "establishments", "n_rooms", "n_beds", "employees", "tourists",
        "altitude", "geography",
    ]
    features_df = df[feature_cols].dropna(subset=["province", "year_month"])
    numeric_cols = features_df.select_dtypes(include=np.number).columns.tolist()
    features_df_norm = normalize_columns(features_df, numeric_cols)
    features_df = pd.concat([features_df, features_df_norm], axis=1)

    return features_df

In [82]:
def recomendar_alternativas(
    features_df: pd.DataFrame, 
    modo: str,
    top_n: int = 3,
    min_crowd_delta: float = 0.25,
    max_user_score_drop: float = 0.15,
    max_temp_deviation_c: float = 3.0,
    sim_geo_bonus: float = 0.08,
    ensure_alternatives: bool = True,    # último recurso si no hay alts
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Recomienda alternativas de provincias.
    """
        # --- Inputs básicos ---
    if modo == "2":
        raw_input = ask("Provincia base (ej: Madrid): ", "Madrid", str).lower()
        provincia_base = validar_provincia(raw_input, features_df["province"])

        
    mes = ask("Mes de viaje (01-12): ", "07", int, lambda x: 1 <= x <= 12)
    anio = ask("Año de viaje (2025-2026): ", "2025", int, lambda x: 2025 <= x <= 2026)
    period = pd.Period(f"{anio}-{mes}", freq="M")
    


    preference = ask("¿Tienes preferencia por algún tipo de evento? (s/n): ", "n", str, lambda x: x in ("s","n"))
    
        # --- Slice de periodo ---
    df_slice = features_df[features_df["year_month"] == period].copy()
    complete_slice_df = df_slice.copy()

    if df_slice.empty:
        raise ValueError(f"No hay datos para {period}")

        # --- Selección de eventos ---
    event_weights = {}
    selected_events = []
    if preference == "s":
        available_event_cols = ["HOBBIES AND GAMES","ARTS AND SOCIETY","SPORTS AND WELLNESS",
                                "FESTIVALS","FOOD","FAMILY"]
        for i,c in enumerate(available_event_cols,1):
            print(f"{i}. {c.capitalize()}")
        while True:
            selection = ask("Selecciona hasta 3 eventos por número (ej:1,2,3): ","")
            try:
                idx = [int(x)-1 for x in selection.split(",") if x.strip()]
                if not idx or len(idx)>3 or any(i<0 or i>=len(available_event_cols) for i in idx):
                    print("⚠️ Entrada inválida.")
                    continue
                break
            except: print("⚠️ Entrada no válida.")
        selected_events = [available_event_cols[i] for i in idx]
        weights_base = np.linspace(1.0,0.5,len(selected_events))
        weights = weights_base / weights_base.sum()
        event_weights = {c:w for c,w in zip(selected_events,weights)}
    else:
        event_weights = {"events_total":1.0}

    # --- Provincia base y tolerancias ---
    if modo == "2":

        prov_map = {p.lower(): p for p in df_slice["province"].unique()}
        if provincia_base not in prov_map:
            raise ValueError(f"Provincia '{provincia_base}' no encontrada")
        
        prov_key = prov_map[provincia_base]
        base_row = df_slice[df_slice["province"] == prov_key].iloc[0]

        tourism_type = base_row["geography"]
        target_temp = base_row["mean_temp"]
        crowd = categorize(base_row["crowd_index"], df_slice["crowd_index"])
        rain = categorize(base_row["rain_index"], df_slice["rain_index"])
        budget = categorize(base_row["price_index"], df_slice["price_index"])

        if tourism_type in ("playa","montaña","urbano","mixto"):
            mask_geo = _geo_mask(df_slice["geography"], tourism_type)
            df_slice = df_slice[mask_geo]
            if df_slice.empty:
                print("EMPTY do to tourism_type")
                return pd.DataFrame(), pd.DataFrame(), []

    else:
        
        tourism_type = ask("Tipo de turismo (playa/montaña/urbano/mixto/n): ", "n",
            str, lambda x: x.lower() in ("playa","montaña","urbano","mixto","n",""))
        
        if tourism_type in ("playa","montaña","urbano","mixto"):
            mask_geo = _geo_mask(df_slice["geography"], tourism_type)
            df_slice = df_slice[mask_geo]
            if df_slice.empty:
                print("EMPTY do to tourism_type")
                return pd.DataFrame(), pd.DataFrame(), []


        temp_min = round(df_slice["mean_temp"].min())
        temp_max = round(df_slice["mean_temp"].max())

        target_temp = ask(
            f"Temperatura deseada ({temp_min}-{temp_max}): ",
            str((temp_min + temp_max) // 2), 
            int,
            lambda x: temp_min <= x <= temp_max
        )
        
        crowd = ask("Tolerancia a multitudes (low/medium/high): ","medium",str, lambda x: x in ("low","medium","high"))
        rain   = ask("Tolerancia lluvia (low/medium/high): ","medium",str, lambda x: x in ("low","medium","high"))
        budget = ask("Presupuesto (low/medium/high): ","medium",str, lambda x: x in ("low","medium","high"))

        # --- Filtrado geográfico estricto ---
        
            # --- Gate de temperatura ---
    df_slice = df_slice[(df_slice["mean_temp"] - target_temp).abs() <= 4.0]

    if df_slice.empty:
        print("EMPTY do to temperature")
        return pd.DataFrame(), pd.DataFrame(), []

        # --- User score ---
    event_signal = sum(df_slice.get(ev, pd.Series(0, index=df_slice.index)) * w
                        for ev, w in event_weights.items())

    offer_cols = ["establishments", "n_beds", "employees"]

    event_index = norm_0_1(np.log1p(event_signal.astype(float)))
    temp_score = np.exp(-((df_slice["mean_temp"] - float(target_temp)).abs() / 4.0))
    offer_score = norm_0_1(df_slice[offer_cols].sum(axis=1))

    crowd_penalty = (1 - df_slice["crowd_index"])
    price_penalty = (1 - df_slice["price_index"])
    rain_penalty  = (1 - df_slice["rain_index"])

    tol_weights = {
        "crowd":{"low":0.5,"medium":0.30,"high":0}.get(crowd,0.30),
        "rain":{"low":0.5,"medium":0.30,"high":0}.get(rain,0.30),
        "price":{"low":0.5,"medium":0.30,"high":0.1}.get(budget,0.30)
        }
    
    tol_comp = (
        tol_weights["crowd"] * crowd_penalty +
        tol_weights["price"] * price_penalty +
        tol_weights["rain"]  * rain_penalty
    )

    base_user_score = (
        0.30 * temp_score +
        0.25 * event_index +
        0.25 * tol_comp +
        0.20 * offer_score
    )

    df_slice["user_score"] = base_user_score
    complete_slice_df["user_score"] = base_user_score

        # --- Top base ---
    if modo == "2":
        top_base = df_slice[df_slice["province"]==prov_key]
        top_set = {prov_key}
    else:
        top_base = df_slice.nlargest(top_n,"user_score")
        top_set = set(top_base["province"])

    columns_to_keep = [c for c in top_base.columns if c.endswith("_index")]
    alt_blocks = []

    # Definimos los intentos de relajación como tuplas de parámetros
    relaxations = [
        (min_crowd_delta, max_temp_deviation_c, max_user_score_drop, True),        # Estricto
        (min_crowd_delta*0.75, max_temp_deviation_c+1, max_user_score_drop+0.05, True),    # Relajación 1
        (min_crowd_delta*0.5, max_temp_deviation_c+2, max_user_score_drop+0.10, True)  # Relajación 2
    ]

    for _, base in top_base.iterrows():
        cand = pd.DataFrame()
        
        # Intentos progresivos
        for crowd_delta, temp_dev, user_drop, require_geo in relaxations:

            M = df_slice[columns_to_keep].copy().to_numpy()
            sim_matrix = cosine_similarity(np.nan_to_num(M, nan=0.0, posinf=0.0, neginf=0.0))

            cand = _alts_once(
                df_slice, base, sim_matrix, tourism_type, sim_geo_bonus, 
                top_set, target_temp, crowd_delta, temp_dev, user_drop, require_geo
            )

            cand = cand[~cand["province"].isin(top_set)]

            if not cand.empty:
                break
        
        # Último recurso si ensure_alternatives=True
        if (cand.empty or len(cand) < top_n) and ensure_alternatives:
            # Inicializamos parámetros relajados
            crowd_relax = 0.25
            temp_relax = 3
            user_drop_relax = 0.15
            step = 0.05  # incremento gradual de user_drop
            max_iter = 20  # evitar bucles infinitos

            M = complete_slice_df[columns_to_keep].copy().to_numpy()
            sim_matrix = cosine_similarity(np.nan_to_num(M, nan=0.0, posinf=0.0, neginf=0.0))

            for _ in range(max_iter):

                cand = _alts_once(
                    complete_slice_df, base, sim_matrix, tourism_type, sim_geo_bonus, 
                    top_set, target_temp, crowd_relax, temp_relax, user_drop_relax, require_geo=False
                )

                cand = cand[~cand["province"].isin(top_set)]

                if not cand.empty and len(cand) >= top_n:
                    cand["note"] = "(relaxed)"
                    break

                # Relajamos progresivamente los parámetros
                crowd_relax *= 0.9
                temp_relax += 1
                user_drop_relax += step
        
        if not cand.empty:
            alt_blocks.append(cand)

    # Concatenamos y limitamos a max_alternatives
    if alt_blocks:
        alternatives = pd.concat(alt_blocks, ignore_index=True).drop_duplicates("province")
        alternatives = alternatives.sort_values("similarity", ascending=False).head(top_n).reset_index(drop=True)
    else:
        alternatives = pd.DataFrame()

    # --- Return según modo ---
    if modo == "2":
        # Modo provincia base
        if alternatives.empty:
            print("\n(No hay alternativas suficientemente similares a la provincia base.)")
            return prov_key, None, selected_events  # devolvemos None en alternativas
        else:
            print("\n=== Alternativas menos masificadas ===")
            for r in alternatives["province"]:
                print(f"- {r}")
            
        return prov_key, alternatives, selected_events
    
    else:
        
        # Modo top destinos
        if top_base.empty:
            print("⚠️ No hay resultados para ese mes.")
            return None, None, selected_events  # devolvemos None en todo
        else:
            print("\n=== Top destinos por tus gustos ===")
            for _, r in top_base.iterrows():
                print(f"- {r['province']}")

        if alternatives.empty:
            print("\n(No hay alternativas menos masificadas suficientemente similares.)")
            return top_base, None, selected_events
        else:
            print("\n=== Alternativas menos masificadas ===")
            for r in alternatives["province"]:
                print(f"- {r}")

            return top_base, alternatives, selected_events



In [9]:
def recomendar_actividades(df, provincia, categorias=None, top_n=3):
    df_filtrado = df[df["NOMBRE_PRO"].str.lower() == provincia.lower()].copy()

    # Filtrar por categorías si el usuario especifica
    if categorias:
        categorias = [c.lower() for c in categorias]
        df_filtrado = df_filtrado[
            df_filtrado["CATEGORIAS"].apply(
                lambda x: any(cat in x.lower() for cat in categorias)
            )
        ]

    # Calcular promedio global
    C = df["RATING"].mean()

    # Calcular weighted rating SOLO si hay reviews
    def calcular_score(row):
        if row["REVIEWS_COUNT"] == 0 or pd.isna(row["RATING"]):
            return -1  # penalización dura
        v = row["REVIEWS_COUNT"]
        R = row["RATING"]
        return (v / (v + 50)) * R + (50 / (v + 50)) * C  # fórmula IMDB

    df_filtrado["score"] = df_filtrado.apply(calcular_score, axis=1)

    # Ordenar por score y seleccionar top_n
    recomendadas = df_filtrado.sort_values("score", ascending=False).head(top_n)

    return recomendadas[["NOMBRE_PRO", "TITULO", "RATING", "REVIEWS_COUNT","score","LINK"]]


In [62]:
def unificado(df_actividades, features_df, top_n_actividades=3):

    print("\n¡Bienvenido al sistema de recomendaciones de viajes!\n")
    print("1) Recomendación automática: te sugerimos destinos según tus preferencias.")
    print("2) Selección de provincia base: tú eliges una provincia y mostramos similares.\n")

    while (modo := input("Ingresa 1 o 2 según el modo que deseas usar: ").strip()) not in ("1", "2"):
        print("⚠️ Entrada no válida. Por favor ingresa 1 o 2.")

    print(f"\nHas elegido {'la recomendación automática' if modo=='1' else 'la selección de provincia base'}.\n")

    result = recomendar_alternativas(features_df, modo)

    top_df, provincia_base, alt_df, selected_events = (
        (result[0], None, result[1], result[2]) if modo == "1"
        else (None, result[0], result[1], result[2])
    )

    top_provincias = top_df["province"].tolist() if top_df is not None else []
    alt_provincias = alt_df["province"].tolist() if alt_df is not None else []


    def mostrar_actividades(provincias, titulo, skip_base=False):
        print(f"\n=== {titulo} ===")
        for prov in provincias:
            if skip_base and prov == provincia_base:
                continue
            if prov.upper() not in df_actividades["NOMBRE_PRO"].str.upper().values:
                print(f"\n⚠️ No hay actividades en {prov}")
                continue
            actividades = recomendar_actividades(df_actividades, prov, categorias=selected_events, top_n=top_n_actividades)
            if actividades.empty:
                print(f"\n⚠️ No hay actividades encontradas en {prov}")
            else:
                print(f"\n--- {prov} ---")
                print(actividades)

    if provincia_base:
        mostrar_actividades([provincia_base], f"ACTIVIDADES EN {provincia_base.upper()}")
    else:
        mostrar_actividades(top_provincias, "ACTIVIDADES EN PROVINCIAS PRINCIPALES")
    if alt_df is not None:
        mostrar_actividades(alt_provincias, "ACTIVIDADES EN PROVINCIAS SIMILARES / ALTERNATIVAS", skip_base=True)
    


In [11]:
experiencias_df, master_forecasted_df = load_data_from_snowflake()
province_month_df = build_province_month_features(master_forecasted_df)
print("✅ PROVINCE_MONTH listo")

✅ Conectado a Snowflake
✅ PROVINCE_MONTH listo


In [90]:
unificado(experiencias_df, province_month_df)


¡Bienvenido al sistema de recomendaciones de viajes!

1) Recomendación automática: te sugerimos destinos según tus preferencias.
2) Selección de provincia base: tú eliges una provincia y mostramos similares.


Has elegido la selección de provincia base.


=== Alternativas menos masificadas ===
- Sevilla
- Salamanca
- Córdoba

=== ACTIVIDADES EN MADRID ===

--- Madrid ---
   NOMBRE_PRO                                             TITULO  RATING  \
1      Madrid          Madrid - Entradas al Parque Warner Madrid     4.7   
44     Madrid     Madrid - Entradas a la Playa del Parque Warner     4.8   
5      Madrid  Madrid - Entradas al Parque de Atracciones de ...     4.7   

    REVIEWS_COUNT     score                                               LINK  
1             644  4.685037  https://www.tuimusement.com/es/espana/madrid/p...  
44             44  4.636339  https://www.tuimusement.com/es/espana/madrid/p...  
5              86  4.623646  https://www.tuimusement.com/es/espana/madrid/p..